In [17]:
import pandas as pd
import os

articles_csv='data/misq_1977_2017_march.csv'
articles_file='articles.txt'
authors_file='authors.txt'
keywords_file='keywords.txt'
try:
    os.remove(authors_file)
except OSError:
    pass
try:
    os.remove(articles_file)
except OSError:
    pass
try:
    os.remove(keywords_file)
except OSError:
    pass

df=pd.read_csv(articles_csv,dtype='str')
df.DE.fillna('',inplace=True)
articles_file=open(articles_file,'a')
authors_file=open(authors_file,'a')
keywords_file=open(keywords_file,'a')
all_authors=[]
all_keywords=[]

def addAuthor(author_name):
    if author_name in all_authors:
		return
    all_authors.append(author_name)    	
    text='''    
---start---
\'\'\'this_title\'\'\'
{{author papers}}
[[Category:author]]
---end---
'''
    text= text.replace('this_title',author_name)
    authors_file.write(text)
    

def addKeyword(keyword):
    if keyword in all_keywords:
		return
    all_keywords.append(keyword)    	
    text='''
---start---
\'\'\'this_title\'\'\'
{{keyword papers}}
[[Category:keyword]]
---end---
'''
    text= text.replace('this_title',keyword)
    keywords_file.write(text)
    

def addArticle(paper):
    authors_list=paper['authors'].split(';') 
    i=0
    authors_text=''
    for author in authors_list:
        author_name_split=reversed(author.strip().split(','))
        author_name=' '.join(name.strip() for name in author_name_split)
        addAuthor(author_name)
        authors_text=authors_text+ author_name +','
    keywords_list=paper['keywords'].split(';') 
    i=0
    keywords_text=''
    for keyword in keywords_list:
        keyword=keyword.strip()
        if keyword != '':
            addKeyword(keyword)
            keywords_text=keywords_text+ keyword+','
        
    text=''' 
---start---
\'\'\'this_title\'\'\'
{{header}}
{{article
|author= this_author
|source= this_source
|year= this_year
|volume=this_volume
|issue=this_issue
|page=this_page
|abstract = this_abstract
|keyword = this_keyword
}}
{{topics}}
{{theories}}
{{methods}}
{{footer}}
[[Category:article]]
---end---
'''
    text= text.replace('this_title',paper['title'])
    text= text.replace('this_author',authors_text)
    text= text.replace('this_source',paper['source'])
    text= text.replace('this_year',paper['year'])
    text= text.replace('this_volume',paper['volume'])
    text= text.replace('this_issue',paper['issue'])
    text= text.replace('this_page',paper['page'])
    text= text.replace('this_abstract',paper['abstract'])
    text= text.replace('this_keyword',keywords_text)
    articles_file.write(text)




In [18]:
df

,Unnamed: 0,TI,AB,AF,PY,VL,IS,DE,BP-EP,SO
0,0,1985 Opinion Survey of MIS Managers: Key Issues,This article identifies the issues which MIS m...,"Curt Hartog, Martin Herbert",1986,10,4,"MIS managers, opinion survey, aligning MIS, en...",351-361,MIS Quarterly
1,1,A Balanced Orientation for the Information Sys...,The orientation of an information systems depa...,"William M. Taggart, Valdur Silbey",1979,3,2,"information systems department, user orientati...",21-33,MIS Quarterly
2,2,A Behavioral Analysis for Planning MIS Impleme...,Planning for a successful MIS implementation r...,Mary Lippitt Nichols,1981,5,1,"implementation, potential adopters, behavioral...",57-66,MIS Quarterly
3,3,A Company/University Joint Venture to Build a ...,A joint venture between a university-based res...,"John R. Weitzel, Kenneth R. Andrews",1988,12,1,"System development, insurance claims processin...",23-34,MIS Quarterly
4,4,A Comparative Examination of Systems Analysis ...,Though many analysis tools and techniques are ...,Mel A. Colter,1984,8,1,"analysis techniques, structured analysis, trad...",51-66,MIS Quarterly
5,5,"A Comparison of Judgement, Skills, and Prompti...",Arguments have been made supporting internal a...,"Severin V. Grabski, J. Hal Reneau, Stephen G. ...",1987,11,2,"Internal controls, internal control evaluation...",151-161,MIS Quarterly
6,6,A Comprehensive Conceptualization of Post-Adop...,"For the last 25 years, organizations have inve...","Jon (Sean) Jasperson, Pamela E. Carter, Robert...",2005,29,3,"IT adoption, IT use, post-adoptive behavior, I...",525-557,MIS Quarterly
7,7,A Confessional Account of an Ethnography About...,Information systems research has traditionally...,Ulrike Schultze,2000,24,1,"Social science, ethnography, evaluation criter...",3-41,MIS Quarterly
8,8,A Configural Approach to Coordinating Expertis...,Despite the recognition of how important exper...,"Srinivas Kudaravalli, Samer Faraj, Steven L. J...",2017,41,01,"Software development, software teams, expertis...",43-64,MIS Quarterly
9,9,A Confirmatory Factor Analysis of the End-User...,The dimensionality of the user satisfaction co...,"William Doll, Xia, Torkzadeah",1994,18,4,"end-user computing, IS success, IS research me...",MIS Quarterly,NaN


In [16]:
for i,row in df.iterrows():
    if i > 10 : break
    paper={
    'title':row['TI'],
    'authors':row['AF'],
    'source':row['SO'],
    'keywords':row['DE'],
    'abstract':row['AB'],
    'year':row['PY'],
    'volume':row['VL'],
    'issue':row['IS'],
    'page':row['BP-EP'],
        }
    print paper['title']

    addArticle(paper)

1985 Opinion Survey of MIS Managers:  Key Issues
A Balanced Orientation for the Information Systems Manager
A Behavioral Analysis for Planning MIS Implementation
A Company/University Joint Venture to Build a Knowledge-Based System
A Comparative Examination of Systems Analysis Techniques
A Comparison of Judgement, Skills, and Prompting Effects Between Auditors and Systems Analysts
A Comprehensive Conceptualization of Post-Adoptive Behaviors Associated with Information Technology Enabled Work Systems
A Confessional Account of an Ethnography About Knowledge Work
A Configural Approach to Coordinating Expertise in Software Development Teams
A Confirmatory Factor Analysis of the End-User Computing Satisfaction Instrument


TypeError: expected a string or other character buffer object

In [7]:
articles_file.close()
authors_file.close()
keywords_file.close()